# Feature Exportation for the Dataset
This script reads a given dataset, converts to S^2 features and exports them all.

In [1]:
from data_source import DataSource
from visualize import Visualize
from sphere import Sphere
from model import Model
from loss import TripletLoss, ImprovedTripletLoss
from training_set import TrainingSet
from average_meter import AverageMeter
from data_splitter import DataSplitter
from feature_set import FeatureSet

import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

import sys
import time
import math
import numpy as np
import pandas as pd
import open3d as o3d
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

from tqdm.auto import tqdm
from scipy import spatial
%reload_ext autoreload
%autoreload 2

## Read initial dataset and compute feature vectors

In [2]:
n_data = 12000
skip_between = 0
ds = DataSource('/media/scratch/berlukas/spherical/training', n_data, skip_between)
ds.load(n_data)
n_data = len(ds.anchors)

Loading anchors from:	 /media/scratch/berlukas/spherical/training/anchor/
Loading positives from:	 /media/scratch/berlukas/spherical/training/positive/
Loading negatives from:	 /media/scratch/berlukas/spherical/training/negative/
Done loading dataset.
	Anchors total: 		12000
	Positives total: 	12000
	Negatives total: 	12000


In [3]:
restore = False
bandwidth = 100
training_set = TrainingSet(ds, restore, bandwidth)
print("Total size: ", len(training_set))

Generating anchor spheres



Generating positive spheres



Generating negative spheres



Generated features
Total size:  12000


## Export the generated feature vectors as PLYs

In [4]:
export_path = '/media/scratch/berlukas/spherical/training_exported_features'
training_set.exportGeneratedFeatures(export_path)

# Testing

The following is not part of the exportation anymore. 

## Load the exported features 

In [14]:
import pandas as pd
df = pd.read_csv(f'{export_path}/anchor/000000000000099.csv')
np.array(df.intensities).reshape([200,200])
np.array(df.ranges).reshape([200,200])

array([[0.00195312, 0.00195312, 0.00195312, ..., 0.        , 0.        ,
        0.        ],
       [0.00195312, 0.00195312, 0.00195312, ..., 0.        , 0.        ,
        0.        ],
       [0.00195312, 0.00195312, 0.00195312, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [29]:
feature_set = FeatureSet(export_path, bandwidth)
print("Total size: ", len(feature_set))

first_anchor=feature_set.anchor_features[0]
print("First anchor shape:", first_anchor.shape)

Loading anchor features from:  /media/scratch/berlukas/spherical/training_exported_features/anchor/*.csv



Loading positive features from:  /media/scratch/berlukas/spherical/training_exported_features/positive/*.csv



Loading negative features from:  /media/scratch/berlukas/spherical/training_exported_features/negative/*.csv



Total size:  100
First anchor shape: (2, 200, 200)
